In [4]:
import time

notebook_start_time = time.time()

# Set up environment

In [ ]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

# Inference pipeline: Deploying and testing the inference pipeline 

In this notebook, we will dig into the inference pipeline and deploy it to Hopsworks as a real-time service.

## 📝 Imports

In [14]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
project, fs = hopsworks_integration.get_feature_store()

# Deploying the ranking inference pipeline
You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it:

In [ ]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(
    project=project
)

Now, we have to explicitly start the deployment:

In [ ]:
ranking_deployment.start()

In [ ]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [19]:

# get Hopsworks Model Serving
ms = project.get_model_serving()

# get deployment object
ranking_deployment = ms.get_deployment("ranking")
# get Hopsworks Model Registry
mr = project.get_model_registry()
  
# get model
model = mr.get_model(ranking_deployment.model_name, ranking_deployment.model_version)


In [20]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

Let's define a dummy test example to test our ranking deployment (only the `user_id` has to match):

In [ ]:
test_ranking_input = [
        {
            "user_id": "a926fcb4-fcb9-461c-bcf3-94e99087488a",
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ]
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve artwork ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

Check logs in case of failure:

In [ ]:
ranking_deployment.get_logs(component="transformer", tail=200)

# Deploying the query inference pipeline

In [ ]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy(project=project)
)

In [ ]:
query_model_deployment.start()

In [ ]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer", tail=20)

# <span style="color:#ff5f27"> Stopping the Hopsworks deployments </span>
Stop the deployment when you're not using it.

In [ ]:
ranking_deployment.stop()
query_model_deployment.stop()